# 1. 기존 데이터 프레임

In [4]:
 pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
from config import DB_CONFIG

# MySQL 연결
conn = pymysql.connect(
    host=DB_CONFIG['host'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password'],
    database=DB_CONFIG['database'],
    port=DB_CONFIG['port'],
    charset='utf8mb4'
) 

In [7]:
# MySQL에서 데이터 가져오기
conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME)
car_siren = f"SELECT * FROM merge_car_siren"
car_horn = f"SELECT * FROM merge_car_horn"
car_driving = f'SELECT * FROM merge_car_driving_sound'
motorcycle_horn  = f'SELECT * FROM merge_motorcycle_horn'
motorcycle_driving  = f'SELECT * FROM merge_motorcycle_driving_sound'

car_siren_df =  pd.read_sql(car_siren, conn)
car_horn_df = pd.read_sql(car_horn, conn) 
car_driving_df = pd.read_sql(car_driving, conn) 
motorcycle_horn_df= pd.read_sql(motorcycle_horn, conn) 
motorcycle_driving_df = pd.read_sql(motorcycle_driving, conn) 

conn.close()

/tmp/ipykernel_23260/1700635407.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_siren_df =  pd.read_sql(car_siren, conn)
/tmp/ipykernel_23260/1700635407.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_horn_df = pd.read_sql(car_horn, conn)
/tmp/ipykernel_23260/1700635407.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_driving_df = pd.read_sql(car_driving, conn)
/tmp/ipykernel_23260/1700635407.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI o

In [8]:
print(f'car_siren 데이터 개수: {car_siren_df.shape[0]}')
print(f'car_horn 데이터 개수: {car_horn_df.shape[0]}')
print(f'car_driving 데이터 개수: {car_driving_df.shape[0]}')
print(f'motorcycle_horn 데이터 개수: {motorcycle_horn_df.shape[0]}')
print(f'motorcycle_driving 데이터 개수: {motorcycle_driving_df.shape[0]}')

car_siren 데이터 개수: 1990
car_horn 데이터 개수: 3189
car_driving 데이터 개수: 1682
motorcycle_horn 데이터 개수: 4560
motorcycle_driving 데이터 개수: 4735


In [9]:
#데이터프레임 병합 
df_combined = pd.concat([car_horn_df, car_siren_df,car_driving_df,motorcycle_horn_df,motorcycle_driving_df], ignore_index=True)

print(f'df_combined 행 개수: {len(df_combined)}') 
df_combined.head()

df_combined 행 개수: 16156


,fileName,labelName,area_start,area_end,category_01,category_02,category_03,decibel,soundQuality,subCategory,...,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,1.car_horn_30849_1.wav,1.car_horn_30849_1.wav,2.0,11.048,교통소음,자동차,차량경적,68,정상,소형차경적,...,37.183453,-10.836115,25.585962,0.232516,5.053774,12.176615,4.831705,-8.831820,10.174751,2.365335
1,1.car_horn_30865_1.wav,1.car_horn_30865_1.wav,2.0,10.672,교통소음,자동차,차량경적,75,정상,소형차경적,...,10.472818,-25.531940,-0.819976,-21.547447,16.138535,-0.490012,0.726548,-9.243734,-1.005557,-10.407813
2,1.car_horn_30869_1.wav,1.car_horn_30869_1.wav,2.0,12.245,교통소음,자동차,차량경적,68,정상,소형차경적,...,-2.516794,-12.093688,21.846487,10.806868,14.971516,9.503950,15.538783,15.781241,0.750223,-8.508451
3,1.car_horn_30874_1.wav,1.car_horn_30874_1.wav,2.0,10.289,교통소음,자동차,차량경적,69,정상,소형차경적,...,18.060278,-17.298338,28.503597,14.064431,7.452422,12.441176,16.612614,2.910048,-6.256705,-6.246398
4,1.car_horn_59_1.wav,1.car_horn_59_1.wav,2.0,4.029,교통소음,자동차,차량경적,107,정상,소형차경적,...,21.661055,-9.801945,12.513287,2.843365,8.194932,7.828580,8.889499,5.067267,2.928293,10.400720


In [10]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16156 entries, 0 to 16155
Data columns (total 45 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fileName       16156 non-null  object 
 1   labelName      16156 non-null  object 
 2   area_start     16156 non-null  float64
 3   area_end       16156 non-null  float64
 4   category_01    16156 non-null  object 
 5   category_02    16156 non-null  object 
 6   category_03    16156 non-null  object 
 7   decibel        16156 non-null  int64  
 8   soundQuality   16156 non-null  object 
 9   subCategory    16156 non-null  object 
 10  bitRate        16156 non-null  object 
 11  duration       16156 non-null  float64
 12  fileFormat     16156 non-null  object 
 13  fileSize       16156 non-null  int64  
 14  recodingType   16156 non-null  object 
 15  sampleRate     16156 non-null  object 
 16  acqDevice      16156 non-null  object 
 17  acqMethod      16156 non-null  object 
 18  acqTyp

# 2. 기타소음 데이터 추가

In [11]:
import os
import json
import librosa
import numpy as np
import pandas as pd

data_dir = "/home/ubuntu/data/etc_noise_data_test"

# 기존 데이터프레임이 존재하는지 확인
try:
    df_combined
except NameError:
    df_combined = pd.DataFrame()  # 없으면 빈 데이터프레임 생성

json_data = []
for file in os.listdir(data_dir):
    if file.endswith(".json"):
        with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
            data = json.load(f)

            annotations = data.get("annotations", [])
            annotation = annotations[0] if annotations and isinstance(annotations, list) else {}

            # JSON 파일명에서 확장자 제거
            base_name = os.path.splitext(file)[0]

            json_data.append({
                "baseName": base_name,  # 확장자 없는 파일명 저장
                "fileName": data["audio"]["fileName"],
                "labelName": annotation.get("labelName", None),
                "area_start": annotation.get("area", {}).get("start", None),
                "area_end": annotation.get("area", {}).get("end", None),
                "category_01": annotation.get("categories", {}).get("category_01", None),
                "category_02": annotation.get("categories", {}).get("category_02", None),
                "category_03": annotation.get("categories", {}).get("category_03", None),
                "decibel": annotation.get("decibel", None),
                "soundQuality": annotation.get("soundQuality", None),
                "subCategory": annotation.get("subCategory", None),
            })

df_json = pd.DataFrame(json_data)

# WAV 파일에서 MFCC 추출하는 함수
def extract_mfcc(file_path, sr=22050, n_mfcc=13):
    try:
        y, sr = librosa.load(file_path, sr=sr)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs, axis=1)  # MFCC 평균값 반환
    except Exception as e:
        print(f"오류 발생: {file_path} - {e}")
        return [None] * n_mfcc  # 오류 발생 시 None 리스트 반환

wav_data = []
for file in os.listdir(data_dir):
    if file.endswith(".wav"):
        file_path = os.path.join(data_dir, file)
        base_name = "_".join(file.split("_")[:-1])  # "_1.wav" 부분 제거하고 기본 파일명 추출
        mfcc_features = extract_mfcc(file_path)

        wav_data.append({
            "baseName": base_name,  # JSON과 매칭할 기본 파일명 저장
            **{f"mfcc_{i+1}": mfcc for i, mfcc in enumerate(mfcc_features)}
        })

df_wav = pd.DataFrame(wav_data)

# JSON 데이터와 WAV 데이터 병합 (baseName 기준)
df_new = pd.merge(df_json, df_wav, on="baseName", how="left").drop(columns=["baseName"])

# 기존 데이터프레임 df_combined에 추가
df_combined = pd.concat([df_combined, df_new], ignore_index=True)

# 결과 확인
print(df_combined.head())

                 fileName               labelName  area_start  area_end  \
0  1.car_horn_30849_1.wav  1.car_horn_30849_1.wav         2.0    11.048   
1  1.car_horn_30865_1.wav  1.car_horn_30865_1.wav         2.0    10.672   
2  1.car_horn_30869_1.wav  1.car_horn_30869_1.wav         2.0    12.245   
3  1.car_horn_30874_1.wav  1.car_horn_30874_1.wav         2.0    10.289   
4     1.car_horn_59_1.wav     1.car_horn_59_1.wav         2.0     4.029   

  category_01 category_02 category_03  decibel soundQuality subCategory  ...  \
0        교통소음         자동차        차량경적       68           정상       소형차경적  ...   
1        교통소음         자동차        차량경적       75           정상       소형차경적  ...   
2        교통소음         자동차        차량경적       68           정상       소형차경적  ...   
3        교통소음         자동차        차량경적       69           정상       소형차경적  ...   
4        교통소음         자동차        차량경적      107           정상       소형차경적  ...   

      mfcc_4     mfcc_5     mfcc_6     mfcc_7     mfcc_8     mfcc_9 

# 3. 검증

In [12]:
print("기존 df_combined 행 개수:", len(df_combined))

기존 df_combined 행 개수: 16176


In [13]:
df_new

,fileName,labelName,area_start,area_end,category_01,category_02,category_03,decibel,soundQuality,subCategory,...,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,3.항공기_3089.wav,3.항공기_3089_1.wav,2.00,25.776,교통소음,항공기,비행기,57,노이즈,민간항공기,...,1.461566,19.411510,11.984865,-3.360847,7.662577,-8.510960,2.700191,-3.754015,6.892228,-3.945253
1,3.항공기_3096.wav,3.항공기_3096_1.wav,2.00,17.881,교통소음,항공기,비행기,94,노이즈,민간항공기,...,-6.802082,7.256363,19.280977,-3.648937,-8.836905,3.380371,4.751110,-7.645988,1.442360,-0.763103
2,4.열차_3548.wav,4.열차_3548_1.wav,2.00,11.856,교통소음,열차,기차,112,정상,고속열차,...,21.332993,-1.507492,13.797204,-0.270314,6.847430,-5.683336,6.834382,-10.379028,7.244081,-3.634330
3,3.항공기_65672.wav,3.항공기_65672_1.wav,0.15,59.820,교통소음,항공기,헬리콥터,57,정상,민간헬리콥터,...,34.963413,20.036774,23.921940,12.042212,-4.608899,-19.376606,-18.259861,-12.896838,13.406316,-3.509350
4,3.항공기_19365.wav,3.항공기_19365_1.wav,9.95,13.500,교통소음,항공기,헬리콥터,103,노이즈,민간헬리콥터,...,41.136570,-13.978091,39.066151,-4.931933,10.940297,0.089313,12.679196,-3.376623,10.103725,-1.016126
5,3.항공기_3090.wav,3.항공기_3090_1.wav,2.00,15.480,교통소음,항공기,비행기,57,정상,민간항공기,...,30.667246,-17.742580,12.550140,-7.764432,3.100761,-7.845706,-1.308434,-1.989534,0.823317,-3.151010
6,4.열차_3551.wav,4.열차_3551_1.wav,2.00,29.724,교통소음,열차,기차,71,노이즈,고속열차,...,35.827324,-2.804525,17.517084,-2.978310,12.821779,-7.200379,7.810171,-4.413130,7.770541,-5.007020
7,3.항공기_65674.wav,3.항공기_65674_1.wav,0.18,22.270,교통소음,항공기,헬리콥터,57,정상,민간헬리콥터,...,35.596443,19.576885,22.898577,11.796556,-4.401205,-19.465029,-18.008459,-11.952487,14.727086,-1.938721
8,4.열차_3807.wav,4.열차_3807_1.wav,2.03,14.030,교통소음,열차,지하철,77,노이즈,지하철,...,48.374195,-2.558496,27.483530,6.862385,2.403998,7.400999,1.968068,1.723226,-2.758039,1.341943
9,4.열차_3806.wav,4.열차_3806_1.wav,2.00,14.326,교통소음,열차,지하철,88,노이즈,지하철,...,17.062466,29.411341,3.418709,15.215955,11.716969,4.410257,7.352183,1.069671,2.770498,5.077219


# 4. 저장

In [15]:
df_new.to_csv("df_ect_noise_20.csv", index=False)

In [17]:
# df_combined.to_csv("df_combined_ect_20.csv", index=False)